In [ ]:
from pylab import *
from pylab import zeros
import math
import random
import struct
from scipy import signal as sg
import scipy
from scipy import special as sp
import numpy as np
from numpy.fft import rfft, irfft
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from PIL import Image
import cv2
import scipy.io.wavfile as wav
import IPython
from collections import Counter

In [ ]:
''' SE RECOMIENDA BORRAR TODAS LAS VARIABLES ANTES DE EJECUTAR EL CÓDIGO '''

#SE DEFINEN LAS FUNCIONES A UTILIZAR:

def most_frequent(List):

    'Program to find most frequent element in a list'

    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]


In [ ]:
'''
PROCEDIMIENTO PARA WATERMARKING
'''

#PASO 1: CARGA DE AUDIO

audio_location=r'D:\(ruta-archivo)\speech.wav'      # <-- CAMBIAR AQUI LA UBICACION DEL ARCHIVO DE AUDIO ORIGINAL.

fs_audio, audio = wav.read(audio_location)

n=np.arange(0,len(audio))

plt.figure()
plt.plot(n,audio)
plt.ylabel('Amplitud')
plt.xlabel('Tiempo [s]')
plt.title('audio')  
plt.show()

print('frecuencia de muestreo : ' + str(fs_audio))
IPython.display.Audio("speech.wav")                                          # <-- CAMBIAR AQUI EL NOMBRE DEL ARCHIVO DE AUDIO.


In [ ]:
#PASO 2: Cargar la imágen y convertirla a binario [0,1]

'''USAR IMAGENES CUADRADAS'''

image_location=r'D:\(ruta-archivo)\image.jpg' # <-- CAMBIAR AQUI LA UBICACION DEL ARCHIVO DE IMAGEN (WATERMARK).

img = cv2.imread(image_location,0)   
ret, bw_img = cv2.threshold(img,127,255,cv2.THRESH_BINARY) #se pasa la imagen a binarios(blanco y negro)
bw_img=(1/255)*bw_img    #el binario de [0,255] pasa a ser [0,1].
plt.imshow(bw_img)
print('tamaño del watermark : '+ str(bw_img.shape))
alto = (len(bw_img[0]))  
ancho = (len(bw_img[:]))
bits_totales = alto * ancho

In [ ]:
#PASO 3: Se obtiene la matriz STFT del audio a marcar

Ventana ='blackman'                              # <-- CAMBIAR AQUI LA VENTANA STFT DESEADA.
m = 256                                          # <-- CAMBIAR AQUI EL ANCHO DESEADO DE LA VENTANA.
overlap = m // 2
nfft=m
return_onesided = True

f,t,audio_stft = sg.stft(audio,fs=fs_audio,window=Ventana,nperseg=m,nfft=nfft,noverlap=overlap,return_onesided=return_onesided)

fig,ax = plt.subplots()
ax.set_ylabel('Frecuencia [Hz]')
ax.set_xlabel('Tiempo [s]')
ax.pcolormesh(t, f, np.log10(np.abs(audio_stft)))

print(sg.check_NOLA(Ventana,m,overlap))

In [ ]:
#PASO 4: Proceso de Marcado del audio

bps = bits_totales

frame_width = (overlap // bps)        # Ancho del frame

a = 0.015                            # <-- CAMBIAR AQUI EL FACTOR DE FUERZA DESEADO.

watermarkBits = bw_img.flatten()      # Pasamos la imagen de matriz a array.

Dxx = np.zeros(audio_stft.shape,dtype=float64)

noise = np.random.normal(0,0.1,size=(frame_width,len(t)))      #Se genera la matriz con el pseudo ruido.
noise[noise == 0] = 0.05             #Para evitar errores en la detección por división sobre cero, se reemplazan los posibles ceros por otro valor no nulo (en este caso 0.05).

Resultado = audio_stft

ind = np.arange(0, len(audio_stft)-frame_width+1, frame_width, dtype = int)    # Determinamos donde empezamos a fragmentar la STFT

# Por cada frame:
for f in range(len(ind)):

    frame = audio_stft[ind[f]:ind[f]+frame_width]

    u, d, vt = np.linalg.svd(frame,full_matrices=True) # SVD de cada Frame
    d = scipy.linalg.diagsvd(d,len(u),len(vt))

    w = noise

    delta_i = np.zeros(d.shape,dtype=float)
    for i in range( len(d) ):
        delta_i[i]= d[i,i]


    wd=[]
    for i in range( len(d) ):
        bit = watermarkBits[f]
        wd.append(array(d[i]+a*w[i]*bit))

    wd = np.array(wd)
    
    uw, dw, vwt = np.linalg.svd(wd,full_matrices=True)
    dw = scipy.linalg.diagsvd(dw,len(uw),len(vwt))
    
    
    Dxx[ind[f]:ind[f]+frame_width] = np.matmul(np.matmul(uw,d),vwt)         # "Uw", "D" y "Vwt" de cada Frame se guardan en este archivo para la posterior detección.
   
    Resultado[ind[f]:ind[f]+frame_width] = np.matmul(np.matmul(u,dw),vt)          # Frame de STFT marcado.

    
Resultado=np.array(Resultado)

                             

In [ ]:
#PASO 5: Inversa de la STFT para obtener el audio marcado

n_audio_marcado, audio_marcado = sg.istft(Resultado,fs=fs_audio,window=Ventana,nperseg=m,noverlap=overlap,nfft=nfft,input_onesided=return_onesided)    #SALE EL AUDIO MARCADO

plt.figure()
plt.plot(n_audio_marcado,audio_marcado)
plt.ylabel('Amplitud')
plt.xlabel('Tiempo [s]')
plt.title('Audio_Marcado')  
plt.show()


In [ ]:
#PASO 6: Exportamos los resultados

np.savetxt("Dxx.csv", Dxx, delimiter=",")         #EXPORTACION DE MATRIZ Dxx.
np.savetxt("key.csv", noise, delimiter=",")       #EXPORTACION DE CONTRASEÑA.
np.savetxt("bps.csv", np.array([bps,0]), delimiter=",")     #EXPORTACION DE BPS.
np.savetxt("audio_original.csv", audio, delimiter=",")       #EXPORTACION DEL AUDIO ORIGINAL EN FORMATO CSV.
np.savetxt("strength_factor.csv", np.array([a,0]), delimiter=",")    #EXPORTACION DEL FACTOR DE FUERZA.
audio_marcado = audio_marcado/np.max(np.abs(audio_marcado))   
audio_marcado = np.iinfo(np.int16).max*audio_marcado
wav.write('audio_marcado.wav',fs_audio,audio_marcado.astype(np.int16))  #EXPORTACION DEL AUDIO MARCADO.
IPython.display.Audio("audio_marcado.wav")

In [ ]:
'''
PARA PRUEBAS: GENERACION DE FALSA CONTRASEÑA (OPCIONAL)
'''

false_key = np.random.normal(0,0.1,size=(frame_width,len(t)))   
np.savetxt("false_key.csv", false_key, delimiter=",")    #EXPORTACION DE FALSA CONSTRASEÑA.